## Traffic Crash Analysis

### Data importing and pre-processing

In [0]:
import requests
from pyspark.sql import SparkSession

# Make the GET request
resp = requests.get('https://data.cityofchicago.org/resource/85ca-t3if.json?$query=SELECT%20crash_record_id%2C%20crash_date_est_i%2C%20crash_date%2C%20posted_speed_limit%2C%20traffic_control_device%2C%20device_condition%2C%20weather_condition%2C%20lighting_condition%2C%20first_crash_type%2C%20trafficway_type%2C%20lane_cnt%2C%20alignment%2C%20roadway_surface_cond%2C%20road_defect%2C%20report_type%2C%20crash_type%2C%20intersection_related_i%2C%20private_property_i%2C%20hit_and_run_i%2C%20damage%2C%20date_police_notified%2C%20prim_contributory_cause%2C%20sec_contributory_cause%2C%20street_no%2C%20street_direction%2C%20street_name%2C%20beat_of_occurrence%2C%20photos_taken_i%2C%20statements_taken_i%2C%20dooring_i%2C%20work_zone_i%2C%20work_zone_type%2C%20workers_present_i%2C%20num_units%2C%20most_severe_injury%2C%20injuries_total%2C%20injuries_fatal%2C%20injuries_incapacitating%2C%20injuries_non_incapacitating%2C%20injuries_reported_not_evident%2C%20injuries_no_indication%2C%20injuries_unknown%2C%20crash_hour%2C%20crash_day_of_week%2C%20crash_month%2C%20latitude%2C%20longitude%2C%20location%20ORDER%20BY%20crash_date%20DESC%2C%20crash_record_id%20ASC')

# Create a Spark session
spark = SparkSession.builder.appName("SENG550").getOrCreate()
#df2 = spark.read.csv("/FileStore/tables/Traffic_Crashes___Crashes.csv", header=True, inferSchema=True)

# Create a Spark DataFrame from the response text
df2 = spark.read.json(spark.sparkContext.parallelize([resp.text]))


In [0]:
# Show the DataFrame
df2.show(5)

+------------------+------------------+--------------------+----------------+-----------------+----------+-----------+--------------------+--------------------+------------+--------------------+--------------------+---------+--------------------+-------------+--------------+-----------------------+----------------------+---------------------------+-----------------------------+--------------+----------------+----------------------+------------+--------------------+--------------------+-------------+--------------------+---------+--------------+------------------+-----------------------+------------------+--------------------+-----------+--------------------+----------------------+------------------+----------------+-----------+---------+----------------------+--------------------+-----------------+-----------+--------------+-----------------+
|         alignment|beat_of_occurrence|          crash_date|crash_date_est_i|crash_day_of_week|crash_hour|crash_month|     crash_record_id|      

### Create RDD of wanted features

In [0]:
wanted_columns = df2.select("Crash_type","num_units","Weather_condition","Crash_date","Most_severe_injury","Longitude","Latitude")
wanted_columns.show(1)
rdd_of_features = wanted_columns.rdd.map(lambda row:[row[0],row[1],row[2],row[3],row[4],row[5],row[6]])


+--------------------+---------+-----------------+--------------------+--------------------+-------------+------------+
|          Crash_type|num_units|Weather_condition|          Crash_date|  Most_severe_injury|    Longitude|    Latitude|
+--------------------+---------+-----------------+--------------------+--------------------+-------------+------------+
|NO INJURY / DRIVE...|        2|            CLEAR|2023-12-12T02:04:...|NO INDICATION OF ...|-87.704126962|41.795222579|
+--------------------+---------+-----------------+--------------------+--------------------+-------------+------------+
only showing top 1 row



### Remove all rows where the content of one of the fields is unknown

In [0]:
print(rdd_of_features.count())
#row[0] = Crash_type, row[2] = Weather_condition,  row[4]= Most_severe_injury
cleaned_data_rdd = rdd_of_features.filter(lambda row: row[0]!="UNKNOWN"  and row[2]!="UNKNOWN"  and row[4]!="UNKNOWN" and row[5] != None and row[6] != None and row[0] != None and row[1] != None and row[2] != None and row[3] != None and row[4] != None)
print(cleaned_data_rdd.count())

1000
912


### Create Dataframe from RDD and get it ready for regression

In [0]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

cleaned_data_df = spark.createDataFrame(cleaned_data_rdd)

#_1 = Crash_type, _2 = numUnits, _3 = weather, _4 = time, _5 = injury severity, _6 = longitude, _7 = latitude
print(cleaned_data_df.dtypes)
numeric_cols = ["_2", "_6", "_7"]
for col_name in numeric_cols:    
    cleaned_data_df = cleaned_data_df.withColumn(col_name, col(col_name).cast("double"))
print(cleaned_data_df.dtypes)

string_cols = ["_1", "_3", "_4", "_5"]
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(cleaned_data_df) for column in string_cols ]

pipeline = Pipeline(stages=indexers)
indexed_df = pipeline.fit(cleaned_data_df).transform(cleaned_data_df)
indexed_df.show(5)

[('_1', 'string'), ('_2', 'string'), ('_3', 'string'), ('_4', 'string'), ('_5', 'string'), ('_6', 'string'), ('_7', 'string')]
[('_1', 'string'), ('_2', 'double'), ('_3', 'string'), ('_4', 'string'), ('_5', 'string'), ('_6', 'double'), ('_7', 'double')]
+--------------------+---+-----+--------------------+--------------------+-------------+------------+--------+--------+--------+--------+
|                  _1| _2|   _3|                  _4|                  _5|           _6|          _7|_1_index|_3_index|_4_index|_5_index|
+--------------------+---+-----+--------------------+--------------------+-------------+------------+--------+--------+--------+--------+
|NO INJURY / DRIVE...|2.0|CLEAR|2023-12-12T02:04:...|NO INDICATION OF ...|-87.704126962|41.795222579|     0.0|     0.0|   699.0|     0.0|
|NO INJURY / DRIVE...|2.0|CLEAR|2023-12-12T01:30:...|NO INDICATION OF ...|-87.676718345|41.889498193|     0.0|     0.0|   698.0|     0.0|
|INJURY AND / OR T...|2.0|CLEAR|2023-12-11T21:45:...|REP

### Create Labeled Points

Partially taken from lab notebook

In [0]:
indexed_rdd = indexed_df.rdd
print(indexed_rdd.take(1)[0][10])

0.0


In [0]:
from pyspark.mllib.regression import LabeledPoint
def createRDD(values):
    return str(values[6]) +',' +str(values[1]) +',' +str(values[5]) +',' +str(values[7]) +',' +str(values[8]) +',' + str(values[9]) +',' + str(values[10])

def parsePoint(line):
    label_features = line.split(',')
    ret_val = LabeledPoint(label_features[0],label_features[1:])
    return ret_val

indexed_rdd = indexed_df.rdd
nice_rdd = indexed_rdd.map(createRDD)
print(nice_rdd.take(1))
parsedPoints = nice_rdd.map(parsePoint)
firstPoint = parsedPoints.take(1)
firstPointFeatures =firstPoint[0].features 
firstPointLabel = firstPoint[0].label
print (firstPointFeatures, firstPointLabel)
d = len(firstPointFeatures)
print(d)

['41.795222579,2.0,-87.704126962,0.0,0.0,699.0,0.0']
[2.0,-87.704126962,0.0,0.0,699.0,0.0] 41.795222579
6


### Normalize features

Taken from lab notebook

In [0]:
def normalizeFeatures(lp):
    """Normalizes features in the LabeledPoint object lp.

    Args:
        lp - LabeledPoint object 

    Returns:
        LabeledPoint: The object contains the label and the normalized features
    """
    normalizedFeatures = list()
    for i in range(0,len(lp.features)):
        feature = (lp.features[i]-broadcastMean.value[i])/broadcastStdev.value[i]
        normalizedFeatures.insert(i,feature)
    return LabeledPoint(lp.label, normalizedFeatures)
    #normalizedAccidents = (lp.features[0]-broadcastMeanAccidents.value)/broadcastStdevAccidents.value
    #normalizedSnoOnGround = (lp.features[1]-broadcastMeanSnoOnGround.value)/broadcastStdevSnoOnGround.value
    #return LabeledPoint(lp.label,[normalizedAccidents,normalizedSnoOnGround])

def getNormalizedRDD(nonNormalizedRDD): 
    """Normalizes the features of the LabeldPoints contained in nonNormalizedRDD.

    Args:
        nonNormalizedRDD - RDD containing non-normalized features 

    Returns:
        returnRDD: RDD containing normalized features
    """


    meanList = list()
    stdevList = list()
    numFeatures = len(nonNormalizedRDD.take(1)[0].features)
    for i in range(0,numFeatures):
        featureRDD = nonNormalizedRDD.map(lambda lp: lp.features[i])
        featureMean = featureRDD.mean()
        featureStdev = featureRDD.stdev()
        meanList.insert(i,featureMean)
        stdevList.insert(i,featureStdev)
    global broadcastMean 
    broadcastMean = sc.broadcast(meanList)
    global broadcastStdev 
    broadcastStdev = sc.broadcast(stdevList)
    returnRDD = nonNormalizedRDD.map(normalizeFeatures)
    return returnRDD

normalizedSamplePoints = getNormalizedRDD(parsedPoints)
print(normalizedSamplePoints.take(5))

[LabeledPoint(41.795222579, [-0.11729077624163668,-0.40893143042628277,-0.7001400420140046,-0.2740935175684388,1.9165425110497238,-0.4050394488109772]), LabeledPoint(41.889498193, [-0.11729077624163668,0.06470797627852717,-0.7001400420140046,-0.2740935175684388,1.9119730994124526,-0.4050394488109772]), LabeledPoint(41.769434236, [-0.11729077624163668,1.1515108917311718,1.4282856857085708,-0.2740935175684388,1.9074036877751814,2.712226182544385]), LabeledPoint(41.809095821, [-0.11729077624163668,0.8248355916686707,-0.7001400420140046,-0.2740935175684388,1.9028342761379102,-0.4050394488109772]), LabeledPoint(41.915626928, [-0.11729077624163668,-0.45460620756549647,1.4282856857085708,-0.2740935175684388,1.898264864500639,1.153593366866704])]


In [0]:
weights = [.8, .2] # train/test split
seed = 42
parsedTrainData, parsedValData = normalizedSamplePoints.randomSplit(weights,seed)
parsedTrainData.cache()
parsedValData.cache()
nTrain = parsedTrainData.count()
nVal = parsedValData.count()

print(nTrain, nVal, nTrain + nVal)
print(normalizedSamplePoints.count())

708 204 912
912


### Create baseline using the average value

In [0]:
averagelatitude = (parsedTrainData.map(lambda s: s.label)).mean()
print(averagelatitude)

41.85629697301126


In [0]:
import math
def squaredError(label, prediction):
    sqrError = (label-prediction)*(label-prediction)
    return sqrError

def calcRMSE(labelsAndPreds):
    sqrSum = labelsAndPreds.map(lambda s: squaredError(s[0],s[1])).sum()
    return math.sqrt(sqrSum/labelsAndPreds.count())

labelsAndPredsTrain = parsedTrainData.map(lambda s: (s.label,averagelongitude))
rmseTrainBase = calcRMSE(labelsAndPredsTrain)

labelsAndPredsVal = parsedValData.map(lambda s: (s.label,averagelongitude))
rmseValBase = calcRMSE(labelsAndPredsVal)
print(rmseValBase)

0.08663353267833755


### Apply linear regression with weights version one

In [0]:
from pyspark.mllib.linalg import DenseVector
from pyspark.mllib.regression import LinearRegressionWithSGD
# Values to use when training the linear regression model
numIters = 500  # iterations
alpha = 1.0  # step
miniBatchFrac = 1.0  # miniBatchFraction
reg = 1e-1  # regParam
regType = 'l2'  # regType
useIntercept = True  # intercept

In [0]:
firstModel = LinearRegressionWithSGD.train(parsedTrainData,numIters,alpha,miniBatchFrac,initialWeights=None,regParam=reg,regType=regType,intercept=useIntercept)

# weightsLR1 stores the model weights; interceptLR1 stores the model intercept
weightsLR1 = firstModel.weights
interceptLR1 = firstModel.intercept
print(weightsLR1, interceptLR1)

/databricks/spark/python/pyspark/mllib/regression.py:367: FutureWarning: Deprecated in 2.0.0. Use ml.regression.LinearRegression.
  warnings.warn("Deprecated in 2.0.0. Use ml.regression.LinearRegression.", FutureWarning)


[0.08129066491587728,-0.0431210922907528,0.024292820696472303,0.024894161025153475,-0.005776763546226742,0.0023650160069275676] 38.08515647193387


In [0]:
samplePoints = parsedValData.take(5)
print(samplePoints)
for i in range(5):
    samplePrediction = firstModel.predict(samplePoints[i].features)
    print(samplePrediction)

[LabeledPoint(41.932253638, [-0.11729077624163668,0.151743427286575,1.4282856857085708,-0.2740935175684388,1.8662789830397404,1.153593366866704]), LabeledPoint(41.768392247, [-0.11729077624163668,0.17854407821960258,-0.7001400420140046,-0.2740935175684388,1.8525707481279268,-0.4050394488109772]), LabeledPoint(41.807530373, [-0.11729077624163668,-1.0858130395007504,1.4282856857085708,-0.2740935175684388,1.8434319248533844,1.153593366866704]), LabeledPoint(41.876914678, [-0.11729077624163668,-1.1185849931706981,-0.7001400420140046,4.487302444763297,1.8251542783042995,-0.4050394488109772]), LabeledPoint(41.852951173, [-0.11729077624163668,0.8242539587043363,-0.7001400420140046,-0.2740935175684388,-0.6697444756457859,-0.4050394488109772])]
38.08889945869792
38.032431318459984
38.14239622737014
38.207054277114665
38.019158421747335


In [0]:
labelsAndPreds = parsedValData.map(lambda lp: (lp.label,firstModel.predict(lp.features)))
rmseValLR1 = calcRMSE(labelsAndPreds)

print(rmseValBase)
print(rmseValLR1)

0.08663353267833755
3.792155122297883


### Apply linear regression with weights 2

In [0]:
numIters = 1000  # iterations
alpha = 1.0  # step
miniBatchFrac = 0.3  # miniBatchFraction
reg = 1e-1  # regParam
regType = 'l2'  # regType
useIntercept = True  # intercept

In [0]:
secondModel = LinearRegressionWithSGD.train(parsedTrainData,numIters,alpha,miniBatchFrac,initialWeights=None,regParam=reg,regType=regType,intercept=useIntercept)

# weightsLR1 stores the model weights; interceptLR1 stores the model intercept
weightsLR2 = secondModel.weights
interceptLR2 = secondModel.intercept
print(weightsLR2, interceptLR2)

[0.04444208382874957,-0.013566012936012102,0.033738936794535414,0.029195007806519672,-0.09553023188832116,-0.01999563353405005] 38.05148829900105


In [0]:
labelsAndPreds = parsedValData.map(lambda lp: (lp.label,secondModel.predict(lp.features)))
rmseValLR2 = calcRMSE(labelsAndPreds)

print(rmseValBase)
print(rmseValLR2)

0.08663353267833755
3.821409973912287


### Random Forest Version One

In [0]:
from pyspark.mllib.tree import RandomForest
thirdModel = RandomForest.trainRegressor(parsedTrainData, categoricalFeaturesInfo={},
                                      numTrees=8, featureSubsetStrategy="auto",
                                      impurity='variance', maxDepth=5, maxBins=32)



In [0]:
samplePoints = parsedValData.take(5)
print(samplePoints)
for i in range(5):
    samplePrediction = thirdModel.predict(samplePoints[i].features)
    print(samplePrediction)

[LabeledPoint(41.932253638, [-0.11729077624163668,0.151743427286575,1.4282856857085708,-0.2740935175684388,1.8662789830397404,1.153593366866704]), LabeledPoint(41.768392247, [-0.11729077624163668,0.17854407821960258,-0.7001400420140046,-0.2740935175684388,1.8525707481279268,-0.4050394488109772]), LabeledPoint(41.807530373, [-0.11729077624163668,-1.0858130395007504,1.4282856857085708,-0.2740935175684388,1.8434319248533844,1.153593366866704]), LabeledPoint(41.876914678, [-0.11729077624163668,-1.1185849931706981,-0.7001400420140046,4.487302444763297,1.8251542783042995,-0.4050394488109772]), LabeledPoint(41.852951173, [-0.11729077624163668,0.8242539587043363,-0.7001400420140046,-0.2740935175684388,-0.6697444756457859,-0.4050394488109772])]
41.81057953937727
41.86332844764969
41.81603843466649
41.88039262902279
41.86552153138955


In [0]:
import numpy as np
labels = parsedValData.map(lambda x: x.label).collect()
predictions = thirdModel.predict(parsedValData.map(lambda x: x.features)).collect()
rmseDT1 = np.sqrt(np.mean((np.array(predictions)-np.array(labels))**2))

print(rmseValBase)
print(rmseDT1)

0.08663353267833755
0.07960741793617669


### Random Forest Version Two

In [0]:
thirdModel = RandomForest.trainRegressor(parsedTrainData, categoricalFeaturesInfo={},
                                      numTrees=10, featureSubsetStrategy="auto",
                                      impurity='variance', maxDepth=6, maxBins=32)

In [0]:
samplePoints = parsedValData.take(5)
print(samplePoints)
for i in range(5):
    samplePrediction = thirdModel.predict(samplePoints[i].features)
    print(samplePrediction)

[LabeledPoint(41.932253638, [-0.11729077624163668,0.151743427286575,1.4282856857085708,-0.2740935175684388,1.8662789830397404,1.153593366866704]), LabeledPoint(41.768392247, [-0.11729077624163668,0.17854407821960258,-0.7001400420140046,-0.2740935175684388,1.8525707481279268,-0.4050394488109772]), LabeledPoint(41.807530373, [-0.11729077624163668,-1.0858130395007504,1.4282856857085708,-0.2740935175684388,1.8434319248533844,1.153593366866704]), LabeledPoint(41.876914678, [-0.11729077624163668,-1.1185849931706981,-0.7001400420140046,4.487302444763297,1.8251542783042995,-0.4050394488109772]), LabeledPoint(41.852951173, [-0.11729077624163668,0.8242539587043363,-0.7001400420140046,-0.2740935175684388,-0.6697444756457859,-0.4050394488109772])]
41.84329119906854
41.86504485416342
41.85140157775568
41.86297343474248
41.86265478823708


In [0]:
labels = parsedValData.map(lambda x: x.label).collect()
predictions = thirdModel.predict(parsedValData.map(lambda x: x.features)).collect()
rmseDT2 = np.sqrt(np.mean((np.array(predictions)-np.array(labels))**2))

print(rmseValBase)
print(rmseDT2)

0.08663353267833755
0.07740030933230682
